# Weight Forecaster MVP V2

## Setup

In [482]:
import sys
toolpath = '/Users/jamieinfinity/Dropbox/Projects/WeightForecaster/weightforecaster/server/src'
sys.path.append(toolpath)

from wtfc_utils import etl_utils as etl

from sqlalchemy import create_engine

import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import altair as alt
from IPython.display import Image

import ipywidgets as widgets
import bqplot as bq
from bqplot import pyplot as plt

## Gathered Reusable Code

### Model code

In [2]:
# Model v0.1
c_w = 0.9842664081035283
c_c = 0.001965638199353011
c_s = -4.621900527451458e-05
c_0 = -1.2110620297640367
alpha_s = -c_s/c_c
alpha_0 = -c_0/c_c
alpha_w = (1-c_w)/c_c
gamma = -np.log(c_w)
steps_goal = 10000
weight_velocity_goal = -0.5

def w_next_week(w_curr_week, c_next_week, s_next_week):
    return c_0 + c_w*w_curr_week + c_c*c_next_week + c_s*s_next_week

def w_steady_state(c, s):
    return (c - alpha_s*s - alpha_0)/alpha_w

def c_steady_state(w, s):
    return (alpha_s*s + alpha_0 + alpha_w*w)

def w_forecast(t_weeks, w_curr_week, c, s):
    wss = w_steady_state(c, s)
    return (w_curr_week - wss)*np.exp(-gamma*t_weeks) + wss

def w_velocity(w_curr_week, c, s):
    wss = w_steady_state(c, s)
    return gamma*(wss - w_curr_week)
    
# def steps_target(s_goal, s_avg):
#     return max(s_goal, s_avg)

def cals_target(w_vel, w_curr_week, s):
    return alpha_w*(w_vel/gamma + w_curr_week) + alpha_s*s + alpha_0

### UI code

In [738]:
weight_target = 155
steps_target = 10000
steps_min = 4000
weight_target_tol = 1
velocity_target = -0.5
velocity_target_mo = 4.33*velocity_target

# target_label_color = '#07817D'
# target_hit_color = '#1AD6CF'
# target_near_color = '#97EDEA'
# target_miss_color = '#EEB00C'

# target_label_color = '#09678A'
# target_hit_color = '#70A65F'
# target_near_color = '#A3CD97'
target_miss_color = '#C85C46'

target_label_color = '#09678A'
target_hit_color = '#45CCCC'
target_near_color = '#8FE0E0'
# target_miss_color = '#DB9307' #'#EEB00C'



label_style = {
    'font-size':'16px', 
    'font-family':'avenir-heavy', 
    'color':'#bbb',
}
value_style = {
    'font-size':'40px', 
    'font-family':'tahoma', 
    'color':'#000',
}
value_style_sm = {
    'font-size':'34px', 
    'font-family':'tahoma', 
    'color':'#000',
}
value_unit_style = {
    'font-size':'22px', 
    'font-family':'avenir-light', 
    'color':'#bbb',
}

def stringify_style(style):
    res=''
    for key in style.keys():
        res = res + key + ':'
        res = res + str(style[key]) + '; '
    return res

def label_html(text, style, text_format=None):
    s = stringify_style(style)
    if text_format=='float':
        v = '<div style="{0}">{1:.1f}</div>'.format(s, text)
    else:
        v = '<div style="{0}">{1}</div>'.format(s, text)
    return widgets.HTML(
        value=v,
        placeholder='',
        description='',
        layout=widgets.Layout(align_self='center')
    )


##### PANEL 1

def panel_forecast_col1(weight):
    color = assign_velocity_color(weight-weight_target-weight_target_tol, -weight_target_tol)
    label = label_html('TODAY', {**label_style, 'margin-bottom':'15px', 'vertical-align':'top'})
    val_num = label_html(weight, {**value_style, 'margin-bottom':'27px', 'color':color}, text_format='float')
    val_units = label_html('lb', {**value_unit_style, 'margin-left':'5px', 'margin-bottom':'8px'})
    val = widgets.HBox([val_num, val_units],
        layout=widgets.Layout(align_self='center'))
    col = widgets.VBox(
        [
            label,
            val
        ],
        layout=widgets.Layout(align_self='center')
    )
    return col

def panel_forecast_col2_sub(weight, target, months, color='#000'):
    label = label_html(months, {**label_style, 'margin-bottom':'1px'})
    wgt_num = label_html(weight, {**value_style_sm, 'margin-bottom':'10px', 'color':color}, text_format='float')
    target_num = label_html(target, {**value_style_sm, 'margin-bottom':'2px', 'color':target_label_color}, text_format='float')
    col = widgets.VBox(
        [
            label,
            wgt_num,
            target_num
        ],
        layout=widgets.Layout(align_self='center', padding='10px')
    )
    return col

def panel_forecast_col2(weight, proj_wgts, targ_wgts):
    tw1 = targ_wgts[0]
    tw2 = targ_wgts[1]
    if weight<(weight_target+weight_target_tol):
        tw1 = weight
        tw2 = weight
    colors = [(target_hit_color if w<weight_target+weight_target_tol else target_miss_color) for w in proj_wgts]
    ca = panel_forecast_col2_sub(proj_wgts[0], tw1, '1 MONTH', color=colors[0])
    cb = panel_forecast_col2_sub(proj_wgts[1], tw2, '2 MONTHS', color=colors[1])
    
    col_a = widgets.VBox(
        [
            label_html('', {**value_unit_style, 'margin-top':'15px'}),
            label_html('current', {**value_unit_style, 'font-size':'14px', 'margin-top':'15px'}),
            label_html('target', {**value_unit_style, 'font-size':'14px', 'margin-top':'8px'}),
        ],
        layout=widgets.Layout(align_self='center', padding='10px')
    )
    
    col = widgets.HBox([col_a, ca, cb], layout=widgets.Layout(width='350px', padding='20px'))
    return col

def panel_forecast_col3(weight, proj_wgts, targ_wgts):
    pw = [x-weight for x in proj_wgts]
    tw = [x-weight for x in targ_wgts]
    maxw = np.max(np.abs([*pw, *tw]))
    factor = 50
    pw = [maxw/factor]+pw
    tw = [-maxw/factor]+tw
    if weight<(weight_target+weight_target_tol):
        tw = [-maxw/factor, -maxw/factor, -maxw/factor]

    bars = bq.Bars(x=[1, 3, 5], y=[pw, tw],  
                   scales={'x': bq.LinearScale(), 'y': bq.LinearScale(min=1.05*min(-5, np.min(tw), np.min(pw)))}, 
                   default_size=1)
    bars.type = 'grouped'
    pc = assign_velocity_color(pw[-1], tw[-1])
    bars.colors = [pc, target_label_color]
    bar_fig = bq.Figure(marks=[bars], layout=widgets.Layout(align_self='center', width='180px', height='80px'), fig_margin=dict(top=0, bottom=0, left=0, right=0))
    labels = widgets.HBox([
        label_html('TODAY', {**label_style, 'font-size':'12px', 'margin-right':'30px'}),
        label_html('1MO', {**label_style, 'font-size':'12px', 'margin-right':'35px'}),
        label_html('2MO', {**label_style, 'font-size':'12px', 'margin-right':'10px'})
    ], layout=widgets.Layout(align_self='center', padding='5px'))
    col3 = widgets.VBox([labels, bar_fig], layout=widgets.Layout(align_self='center', padding='10px'))
    return col3

def panel_forecast(today_date, data_df):
    yesterday = today_date - datetime.timedelta(days = 1)
    df_today = data_df[data_df.index==pd.to_datetime(today_date)]
    df_yest = data_df[data_df.index==pd.to_datetime(yesterday)]

    w_avg_yest = df_today.w_7day_avg.values[0]
    w_proj_1mo = df_today.Mv1_0_proj_weight_1mo.values[0]
    w_proj_2mo = df_today.Mv1_0_proj_weight_2mo.values[0]
    w_targ_1mo = df_today.Mv1_0_target_weight_1mo.values[0]
    w_targ_2mo = df_today.Mv1_0_target_weight_2mo.values[0]

    col1 = panel_forecast_col1(w_avg_yest)
    col2 = panel_forecast_col2(w_avg_yest, [w_proj_1mo, w_proj_2mo], [w_targ_1mo, w_targ_2mo])
    col3 = panel_forecast_col3(w_avg_yest, [w_proj_1mo, w_proj_2mo], [w_targ_1mo, w_targ_2mo])

    grid = widgets.GridspecLayout(1, 3, width='780px', height='180px')
    grid[0,0] = col1
    grid[0,1] = col2
    grid[0,2] = col3

    return grid


##### PANEL 2

def panel_velocity_col(velocity, label, color='#000'):
    label = label_html(label, {**label_style, 'margin-bottom':'7px'})
    num = label_html(velocity, {**value_style, 'color':color, 'margin-bottom':'8px'}, text_format='float')
    units = label_html('lb/mo', {**value_unit_style, 'font-size':'18px', 'margin-bottom':'5px'})
    col = widgets.VBox(
        [
            label,
            num,
            units
        ],
        layout=widgets.Layout(align_self='center', padding='5px')
    )
    return col

def assign_velocity_color(val, target):
    color = target_miss_color
    if val<target:
        color = target_hit_color
    elif val<0:
        color = target_near_color
    return color

def panel_velocity_bars(date, data_df, w_vel_target):
    # maybe base bar color on Mv1_0_velocity_target, which is 0 if weight is within desired range
    start_date = date - datetime.timedelta(days = 31)
    temp_df = data_df[(data_df.index>pd.to_datetime(start_date)) & (data_df.index<=pd.to_datetime(date))]
    wgt_vel = temp_df.Mv1_0_weight_velocity.values
    date_min = temp_df.index.min()
    date_max = temp_df.index.max()

    x_sc = bq.LinearScale()
    y_sc = bq.LinearScale(min=1.05*min(np.min(wgt_vel),w_vel_target), max=1.05*max(np.max(wgt_vel), 0.5))

    bars = bq.Bars(x=list(range(len(wgt_vel))), y=wgt_vel,  
                   scales={'x': x_sc, 'y': y_sc}, 
                   default_size=1)
    bars.colors = [assign_velocity_color(v, w_vel_target) for v in wgt_vel]

    target_line = bq.Lines(x=[-0.5, len(wgt_vel)-0.5], y=[w_vel_target, w_vel_target],  
                   scales={'x': x_sc, 'y': y_sc}, 
                   default_size=1)
    target_line.colors = [target_label_color]

    bar_fig = bq.Figure(marks=[bars, target_line],
                        layout=widgets.Layout(align_self='center', width='350px', height='100px'), 
                        fig_margin=dict(top=20, bottom=0, left=0, right=0))

    return (bar_fig, [date_min, date_max])

def panel_velocity(today_date, data_df):
#     yesterday = today_date - datetime.timedelta(days = 1)
    df_today = data_df[data_df.index==pd.to_datetime(today_date)]
    w_avg = df_today.w_7day_avg.values[0]
    w_vel_target = velocity_target
    if w_avg < weight_target + weight_target_tol:
        w_vel_target = 0.0
    w_vel_mo = df_today.Mv1_0_weight_velocity.values[0]*4.33
    w_vel_target_mo = w_vel_target*4.33

    col1 = panel_velocity_col(w_vel_target_mo, 'TARGET', color=target_label_color)
    col3 = panel_velocity_col(w_vel_mo, 'TODAY', color=assign_velocity_color(w_vel_mo, w_vel_target_mo))
    
    (bars, dates) = panel_velocity_bars(today_date, data_df, w_vel_target)
    ds = label_html(dates[0].strftime('%b %d'), {**label_style, 'font-size':'10px', 'margin-left':'15px', 'margin-top':'-5px'})
    df = label_html('today', {**label_style, 'font-size':'10px', 'margin-left':'290px', 'margin-top':'-5px'})
    bar_labels = widgets.HBox([ds, df], layout=widgets.Layout(padding='1px'))
    col2 = widgets.VBox([bars, bar_labels], layout=widgets.Layout(width='400px', padding='5px'))

    grid = widgets.GridspecLayout(1, 3, width='780px', height='180px')
    grid[0,0] = col1
    grid[0,1] = col2
    grid[0,2] = col3

    return grid



##### PANEL 3

pos_hack = {
    1: '-11px',
    2: '-21px',
    3: '-32px',
    4: '-42px',
    5: '-53px'
}

def progress_pie(value, threshold, threshold_max=True):
    num_digits = len(str(value))
    right_pos = pos_hack[num_digits]
    color = target_miss_color
    val_prog = value/threshold
    if val_prog>1:
        val_prog = 1
    val_remain = 1 - val_prog
    if (val_prog<1 and threshold_max==True) or (val_prog==1 and threshold_max==False):
        color = target_hit_color    
    pie = bq.Pie(sizes=[val_remain, val_prog],
              colors=["#ddd", color],
              radius=120, inner_radius=100, stroke='#888',
              start_angle=90, end_angle=-90, sort=False
              )
    fig = bq.Figure(marks=[pie],
                    layout=widgets.Layout(align_self='center', width='350px', height='100px'), 
                    fig_margin= dict(top=0, bottom=-110, left=0, right=0)
          )
    fig.layout.width="300px"
    fig.layout.height="150px"


    label = label_html('TODAY', {**label_style, 'margin-bottom':'0px', 'z-index':'1', 'position':'absolute', 'top':'58px', 'right':'-26px'})
    val_num = label_html(value, {**value_style, 'color':color, 'margin-bottom':'0px', 'z-index':'1', 'position':'absolute', 'top':'95px', 'right':right_pos})
    return widgets.VBox([
        label,
        val_num,
        fig
    ], layout=widgets.Layout(align_self='center', padding='0px'))

def assign_bar_color(value, target, threshold_max=True):
    if (value<=target and threshold_max==True) or (value>=target and threshold_max==False):
        return target_hit_color
    else:
        return target_miss_color

def progress_bars(values, targets, val_avg, dates, y_min=0.0, threshold_max=True):
    # maybe draw distinct lines for each day to show that targets varies each day
    # but for now, I don't think the difference will show up very well on given the scale
    vals = np.array(values)
    avg = val_avg

    x_sc = bq.LinearScale()
    y_sc = bq.LinearScale(min=min(y_min, 0.95*np.min(vals)), max=1.05*np.max(vals))
    
    bars = bq.Bars(x=list(range(len(vals))), y=vals,  
                   scales={'x': x_sc, 'y': y_sc}, 
                   default_size=1)
    bars.colors = [assign_bar_color(vals[i], targets[i], threshold_max) for i in range(len(vals))]
    
    target_line = bq.Lines(x=[-0.5, len(vals)-0.5], y=[targets[-1], targets[-1]],  
                   scales={'x': x_sc, 'y': y_sc}, stroke_width=3)
    target_line.colors = [target_label_color]        

    avg_line = bq.Lines(x=[-0.5, len(vals)-0.5], y=[avg, avg],  
                   scales={'x': x_sc, 'y': y_sc}, stroke_width=2, line_style='dashed')
    avg_line.colors = ['#999']   
    
    bar_fig = bq.Figure(marks=[bars, target_line, avg_line],
                    layout=widgets.Layout(align_self='center', width='250px', height='100px'), 
                    fig_margin=dict(top=20, bottom=0, left=0, right=0))
    

    ds = label_html(dates[0].strftime('%b %d'), {**label_style, 'font-size':'10px', 'margin-left':'5px', 'margin-top':'-5px'})
    df = label_html('today', {**label_style, 'font-size':'10px', 'margin-left':'157px', 'margin-top':'-5px'})
    bar_labels = widgets.HBox([ds, df], layout=widgets.Layout(padding='1px'))
    return widgets.VBox([bar_fig, bar_labels], layout=widgets.Layout(align_self='center', width='250px', padding='5px'))    

def labeled_value(label_text, value, value_color='#000'):
    label = label_html(label_text,  {**label_style, 'margin-bottom':'7px'})
    val = label_html(value, {**value_style, 'color':value_color, 'margin-bottom':'0px'})
    return widgets.VBox([
        label,
        val
    ], layout=widgets.Layout(align_self='center', padding='10px'))

def progress_column(header_text, value, val_targets, roll_avg, vals_8d, dates, y_min=0.0, threshold_max=True):
    header = label_html(header_text, {**label_style, 'color':'#666', 'font-size':'22px', 'margin-bottom':'0px'})
    pie = progress_pie(value, val_targets[-1], threshold_max=threshold_max)
    target = labeled_value('TARGET', int(val_targets[-1]), target_label_color)
    avg = labeled_value('7 DAY AVG', roll_avg, '#888')
    prog_bars = progress_bars(vals_8d, val_targets, roll_avg, dates, y_min, threshold_max=threshold_max)
        
    col = widgets.VBox([
        header,
        pie,
        target,
        avg,
        prog_bars
    ], layout=widgets.Layout(align_self='center', padding='20px'))
    return col

def panel_progress(today_date, data_df):
    yesterday = today_date - datetime.timedelta(days = 1)
    df_today = data_df[data_df.index==pd.to_datetime(today_date)]
    df_yesterday = data_df[data_df.index==pd.to_datetime(yesterday)]
    date_8d = today_date - datetime.timedelta(days = 8)
    df_8d = data_df[(data_df.index>pd.to_datetime(date_8d)) & (data_df.index<=pd.to_datetime(today_date))]
    
    wgt = df_today.w_7day_avg.values[0]
    steps = int(round(df_today.steps.values[0]))
    steps_avg = int(round(df_yesterday.s_7day_avg.values[0]))
    steps_8d = df_8d.steps.values
    steps_targ_8d = df_8d.Mv1_0_target_steps_day.values
    calories = int(round(df_today.calories.values[0]))
    cal_avg = int(round(df_yesterday.c_7day_avg.values[0]))
    cals_8d = df_8d.calories.values
    cals_targ_8d = df_8d.Mv1_0_target_cals_day.values
    if wgt < weight_target + weight_target_tol:
        w_vel_target = 0.0
    else:
        w_vel_target = weight_velocity_goal
    calories_target = int(np.round(cals_target(w_vel_target, wgt, steps)))
    
    grid = widgets.GridspecLayout(1, 2, width='780px', height='620px')
    grid[0,0] = progress_column('CALORIES', calories, cals_targ_8d, cal_avg, cals_8d, df_8d.index, 1500.0, threshold_max=True)
    grid[0,1] = progress_column('STEPS', steps, steps_targ_8d, steps_avg, steps_8d, df_8d.index, 5000.0, threshold_max=False)
    return grid

def weight_history(dates, vals_daily, vals_avg, target, proj_weight_2mo, targ_weight_2mo):
    vals_target = [target for i in range(len(dates))]

    today_date = dates[-1]
    today_wgt = vals_avg[-1]
    future_date = dates[-1] + datetime.timedelta(days = 91)
    
    x_sc = bq.DateScale(min=np.min(list(dates)), max=future_date)
    y_sc = bq.LinearScale(min=150.0, max=np.max(vals_daily[~np.isnan(vals_daily)]))
    ax_x = bq.Axis(scale=x_sc, grid_lines='solid', tick_format='%b')
    ax_y = bq.Axis(scale=y_sc, orientation='vertical')

    pts = bq.Scatter(x=dates, y=vals_daily,  
                   scales={'x': x_sc, 'y': y_sc})
    pts.colors = ["#000"]
    pts.opacities = [0.13]
    
    target_line = bq.Lines(x=(list(dates)+[future_date]), y=(vals_target+[target]), 
                   scales={'x': x_sc, 'y': y_sc}, stroke_width=1, line_style='dashed')
    target_line.colors = [target_label_color]        

    avg_line = bq.Lines(x=dates, y=vals_avg,  
                   scales={'x': x_sc, 'y': y_sc}, stroke_width=2)
    avg_line.colors = ['#000']   
    
    proj_line = bq.Lines(x=[today_date, future_date], y=[today_wgt, proj_weight_2mo], 
                   scales={'x': x_sc, 'y': y_sc}, stroke_width=2, line_style='solid')
    proj_line.colors = [target_hit_color if proj_weight_2mo<today_wgt else target_miss_color]

    proj_line_targ = bq.Lines(x=[today_date, future_date], y=[today_wgt, targ_weight_2mo], 
                   scales={'x': x_sc, 'y': y_sc}, stroke_width=2, line_style='solid')
    proj_line_targ.colors = [target_label_color]    
    
    fig = bq.Figure(marks=[pts, avg_line, target_line, proj_line_targ, proj_line], axes=[ax_x, ax_y], 
                    layout=widgets.Layout(align_self='center', width='750px', height='400px'),
                    fig_margin=dict(top=20, bottom=30, left=50, right=50))
    
    return fig

def panel_history(today_date, data_df):
    date_initial = today_date - datetime.timedelta(days = 456)
    df = data_df[(data_df.index>pd.to_datetime(date_initial)) & (data_df.index<=pd.to_datetime(today_date))]

    return weight_history(pd.to_datetime(df.index), df.weight.values, df.w_7day_avg, weight_target, df.Mv1_0_proj_weight_2mo.iloc[-1], df.Mv1_0_target_weight_2mo.iloc[-1])
    

def app(today_date, data_df):
    grid = widgets.GridspecLayout(4,1, width='780px', height='1500px')

    p_prog = widgets.VBox([
        label_html('Daily Progress', {**label_style, 'font-family':'avenir', 'color':'#555', 'font-size':'26px', 'margin-top':'10px', 'align-content':'left', 'width':'750px'}),
        panel_progress(today_date, data_df)
    ], layout=widgets.Layout(align_self='center', border='1px solid #bbb', height='621px'))    

    p_vel = widgets.VBox([
        label_html('Weight Velocity', {**label_style, 'font-family':'avenir', 'color':'#555', 'font-size':'26px', 'margin-top':'10px', 'align-content':'left', 'width':'750px'}),
        panel_velocity(today_date, data_df)
    ], layout=widgets.Layout(align_self='center', border='1px solid #bbb'), height='170px') 
    
    p_fcast = widgets.VBox([
        label_html('Weight Forecast', {**label_style, 'font-family':'avenir', 'color':'#555', 'font-size':'26px', 'margin-top':'10px', 'align-content':'left', 'width':'750px'}),
        panel_forecast(today_date, data_df)
    ], layout=widgets.Layout(align_self='center', border='1px solid #bbb', height='225px')) 

    p_hist = widgets.VBox([
        label_html('Weight History', {**label_style, 'font-family':'avenir', 'color':'#555', 'font-size':'26px', 'margin-top':'10px', 'align-content':'left', 'width':'750px'}),
        panel_history(today_date, data_df)
    ], layout=widgets.Layout(align_self='center', border='1px solid #bbb', height='425px')) 
    
    grid[0,0] = p_prog
    grid[1,0] = p_vel
    grid[2,0] = p_fcast
    grid[3,0] = p_hist

    return grid

In [746]:
# app(today, db_df)

## Load Data

In [742]:
server_dir = '/Users/jamieinfinity/Dropbox/Projects/WeightForecaster/weightforecaster/server/'
db_dir = server_dir + 'db/'
db_name = 'weightforecaster'
db_ext = '.db'
db_file_name = db_dir + db_name + db_ext

engine = create_engine('sqlite:///'+db_file_name)

with engine.connect() as conn, conn.begin():
    db_df = pd.read_sql_table('fitness', conn, index_col='date', parse_dates=['date'])
    
# today_offset = 501
# today_offset = 301
# today_offset = 266
# today_offset = 0
# today = datetime.date.today() - datetime.timedelta(days = today_offset)

# today = datetime.datetime.strptime('2020-03-02', '%Y-%m-%d').date()
# today = datetime.datetime.strptime('2020-09-18', '%Y-%m-%d').date()
# today = datetime.datetime.strptime('2020-10-23', '%Y-%m-%d').date()
# today = datetime.datetime.strptime('2020-10-30', '%Y-%m-%d').date() # reveals a bug on proj weight bars (axis limits)
# today = datetime.datetime.strptime('2021-6-15', '%Y-%m-%d').date()
today = datetime.date.today()
# print(today)

# db_df.tail(8)

In [745]:
app(today, db_df)

GridspecLayout(children=(VBox(children=(HTML(value='<div style="font-size:26px; font-family:avenir; color:#555…